<h1 align=center><font size = 5>The Battle of Neighborhoods in New York City</font></h1>

<h1 align=center><font size = 2>Applied Data Science Capstone by IBM/Coursera</font></h1>

# **Objective**

**You will have the opportunity to be as creative as you want and come up with an idea to leverage the Foursquare location data to explore or compare neighborhoods or cities of your choice or to come up with a problem that you can use the Foursquare location data to solve.**

# **Introduction: Background**

**Among all the widely consumed beverages in the world, coffee and tea is consider the top selection. In Canada, according to Coffee Association of Canada the coffee industry is a $6.2 billion industries with over 5,000 independent cafe and coffee shops.
Beside independent coffee shops, you have major chain coffee shop like Starbucks, Country Styles, Second Cup and the biggest of them all Tim Horton's with over 3,800 locations.**


# **Problem**

**Through online research of Coffee and Tea in Canada I discovered my favorite coffee chain shop is not in Canada. My favorit chain is Coffee Beans and Tea Leafs. Coffee Beans and Tea Leaf started out in my home state of California. It has expanded into 32 different countries but not in Canada. The scenario that I have come up with is "What if Coffee Beans and Tea Leaf decided to expand to Vancouver where will the hotspot be?"**

# **Description of the Data**

**I will be using postal codes provide from wikipedia as neighborhood or the hotspot to recommend Coffee Beans and Tea Leafs for there intial location.
I will also be using foursquare data with geolocater and cluster to determine where customer will likely frequent for coffee.**

**1. Link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"**

**Using Postal Codes to define hotspot in Vancouver**

**2. Foursquare.com - category - coffee shop**

**Using foursquare data to fine the hotspot for coffee shop with clusters in Vancouver**

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geocoder

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup # library for extract data from HTML or XML

!pip install folium

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 12.0MB/s ta 0:00:01
     |████████████████████████████████| 102kB 9.6MB/s ta 0:00:011
Libraries imported.


In [16]:
# Load article, turn into soup and get the <table>s.
source = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table_class = 'wikitable sortable .text'
response = requests.get(source)
soup = BeautifulSoup(response.text, 'xml')
list=soup.find('table')


In [17]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
df

,Postalcode,Borough,Neighborhood


In [18]:
# Transforming rows for postcode,borough and neighborhood to df 

for tr_cell in list.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data


In [19]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [14]:
df=df[df['Borough']!='Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [21]:
geo_df.head ()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True) # rename Postal Code to Postalcode
geo_merged = pd.merge(geo_df, df, on='Postalcode') # merge csv with df in part 1 using Postalcode

In [23]:
neighborhoods=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']] # creating a new strin

In [25]:
neighborhoods.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [52]:
CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ' # your Foursquare ID
CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' # your Foursquare Secret
VERSION = '20180604' # Create a function to repeat the same process to all the neighborhoods in Toronto
search_query = 'Coffee Cafe'
categoryId = '4bf58dd8d48988d1e0931735'

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    radius = 500 # define radius
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&&categoryId={}'.format(CLIENT_ID,CLIENT_SECRET,latitudes,longitudes,VERSION,search_query,radius,LIMIT,categoryId)

            
# make the GET request
results = requests.get(url).json()["response"]["venues"]
        
# return only relevant information for each nearby venue
venues_list.append([(name, lat, lng, v['venues']['name'], v['venues']['location']['lat'], v['venues']['location']['lng']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
 

IndentationError: expected an indented block (<ipython-input-52-a9e27a2cbb98>, line 14)

In [ ]:
   return(nearby_venues)

In [37]:
toronto=neighborhoods[neighborhoods['Borough'].str.contains("Toronto")] # new string borough that only contain the word Toronto
toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [38]:
# run the above function on each neighborhood and create a new dataframe called toronto_venues
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'], 
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High